In [66]:
import pandas as pd
from pathlib import Path
import shutil

import PC_Fns
from PC_Fns import PandUPF
import decodeRAW_HK

In [73]:
# First load pickle and create a unedited copy
proc_dir = Path(r"C:\WDir\PanCam EGSE\Logs\FS Calibration Reprocessed\20191011_095418_WAC_R5_-50c_ABSR\PROC")
unproc = "Unproc_HKTM.pickle"
curfile = proc_dir / unproc

RTM = pd.read_pickle(proc_dir / unproc)
curfile.rename(proc_dir / (unproc + ".orig"))

RTM.head()

,Time,RAW,Source
0,2019-10-11\t09:54:35.576,15 07 00 0F 0B DB 02 04 00 00 4D 00 07 18 04 8...,LabView
1,2019-10-11\t09:54:36.576,15 07 00 0F 0B DC 02 04 00 00 4D 00 07 18 04 8...,LabView
2,2019-10-11\t09:54:37.576,15 07 00 0F 0B DD 02 04 00 00 4D 00 07 18 04 8...,LabView
3,2019-10-11\t09:54:38.576,15 07 00 0F 0B DE 02 04 00 00 4D 00 07 18 04 8...,LabView
4,2019-10-11\t09:54:39.576,15 07 00 0F 0B DF 02 04 00 00 4D 00 07 18 04 8...,LabView


In [74]:
# Determine problem range

Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))

TM = pd.DataFrame()
TM['Stat_PIU_En'] = PandUPF(Bin, 'u8', 42, 0)
TM['Stat_PIU_Pw'] = PandUPF(Bin, 'u8', 43, 0)

WACBin, HRCBin = decodeRAW_HK.Determ_CamRes(TM, Bin)
WACBin.head()

127     [53, 3, 0, 15, 12, 90, 2, 4, 0, 0, 61, 0, 7, 2...
1306    [53, 3, 0, 15, 16, 250, 2, 4, 0, 0, 61, 0, 7, ...
1308    [53, 3, 0, 15, 16, 252, 2, 4, 0, 0, 61, 0, 7, ...
1309    [53, 3, 0, 15, 16, 253, 2, 4, 0, 0, 61, 0, 7, ...
1310    [53, 3, 0, 15, 16, 254, 2, 4, 0, 0, 61, 0, 7, ...
Name: RAW, dtype: object

In [75]:
# CamRes bytes at [44:64]
# 3 characters per byte

camresnul = RTM.RAW[127:1306].apply(lambda x: x[:44*3] + x[44*3:64*3].replace('FF', '00') + x[64*3:])

RTM['RAW'].at[127:1305] = camresnul
RTM[1300:].head(20)

,Time,RAW,Source
1300,2019-10-11\t10:16:20.376,15 07 00 0F 10 F4 02 04 00 00 4D 00 07 1A 04 8...,LabView
1301,2019-10-11\t10:16:21.376,15 07 00 0F 10 F5 02 04 00 00 4D 00 07 1A 04 8...,LabView
1302,2019-10-11\t10:16:22.376,15 07 00 0F 10 F6 02 04 00 00 4D 00 07 1A 04 8...,LabView
1303,2019-10-11\t10:16:23.376,15 07 00 0F 10 F7 02 04 00 00 4D 00 07 1A 04 8...,LabView
1304,2019-10-11\t10:16:24.376,35 03 00 0F 10 F8 02 04 00 00 3D 00 07 19 04 8...,LabView
1305,2019-10-11\t10:16:25.376,15 07 00 0F 10 F9 02 04 00 00 4D 00 07 1A 04 8...,LabView
1306,2019-10-11\t10:16:26.376,35 03 00 0F 10 FA 02 04 00 00 3D 00 07 1A 04 8...,LabView
1307,2019-10-11\t10:16:27.376,15 07 00 0F 10 FB 02 04 00 00 4D 00 07 1A 04 8...,LabView
1308,2019-10-11\t10:16:28.376,35 03 00 0F 10 FC 02 04 00 00 3D 00 07 1A 04 8...,LabView
1309,2019-10-11\t10:16:29.377,35 03 00 0F 10 FD 02 04 00 00 3D 00 07 1A 04 8...,LabView


In [76]:
# Check RTM
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))

TM = pd.DataFrame()
TM['Stat_PIU_En'] = PandUPF(Bin, 'u8', 42, 0)
TM['Stat_PIU_Pw'] = PandUPF(Bin, 'u8', 43, 0)

WACBin, HRCBin = decodeRAW_HK.Determ_CamRes(TM, Bin)
WACBin.head()
TM = decodeRAW_HK.DecodeWAC_CamRes(TM, WACBin)

In [77]:
# Finally write new pickle
RTM.to_pickle(proc_dir / unproc)